In [11]:
import numpy as np
import itertools
import pandas as pd
from datetime import datetime, timedelta
import random


# Load the products ,sales , and inventory data into dataframes
accepted_inv = pd.read_csv('accepted_inv.csv')
products = pd.read_csv('Products.csv')
sales_data = pd.read_csv('50000orders.csv')

### Optimization Process Intro:
##### 1/Inventory Filter
##### 2/Combination Generator
##### 3/Shipping Cost Calculator
##### 4/Condtion Cost Calculator
##### 5/Final Optimazition Model


**test order: 83845230**

In [12]:
## Data Cleaning:
now = datetime.now()

# fill all the na in weight with the mean
avg_weight = round(products['Weight'].mean(),4)
products['Weight'].fillna(avg_weight, inplace=True)

# Replacing null values in 'DateLeftInventory' column with the current date and time
accepted_inv['DateLeftInventory'].fillna(now, inplace=True)

# Converting 'DateLeftInventory' and 'DateProcessed' columns to datetime objects
accepted_inv['DateLeftInventory'] = pd.to_datetime(accepted_inv['DateLeftInventory'])
accepted_inv['DateProcessed'] = pd.to_datetime(accepted_inv['DateProcessed'])

# Convert the 'DateOrdered' column in the sales_data dataframe to a datetime format
sales_data['DateOrdered'] = pd.to_datetime(sales_data['DateOrdered'])

#Convert 'Mishawaka' to 'Mish'
sales_data['ShipFromWarehouse'].replace('Mishawaka','Mish', inplace=True)

# select rows where ConditionVariantID is in [1,2,3,4,5,6,7,8,14]
condition = sales_data['ConditionVariantID'].isin([1,2,3,4,5,6,7,8,14])

sales_data = sales_data.drop(sales_data[~condition].index)

# import shipping rate dataframe
shipping_rate = pd.read_csv('USDomesticShippingRates.csv')

# create condition rate dataframe
cond_cost_df = pd.DataFrame({
    'Condition': ['New(1)', 'LikeNew(8/14)', 'VeryGood(4/5)', 'Good(2/3)', 'Acceptable(6/7)'],
    'Cost': [0, 0, 0.25, 0.25, 0]
})


### Query Generator

In [13]:
'''You can enter the order_id manually or automatically select an order_id at random from the sales data. 
   The returned queries contain the order_id and other information corresponding to it, 
   such as 'product_id', 'requested_num', 'order_time', 'requested_condition', 'requested_no_library_mark'
   , 'FromMishZone', 'FromRenoZone', 'weight'
'''

def generate_query(order_id=None):
    # If no order_id is given, randomly select an order ID from the sales data
    if order_id is None:
        # randomly select an order ID
        order_id = random.choice(sales_data['OrderId'].unique())
    else:
        # Check if the order ID exists in the sales data
        if order_id not in sales_data['OrderId'].unique():
            print(f"No such order ID:{order_id}")
    
    # Select the corresponding product IDs based on the order ID
    product_ids = sales_data.loc[sales_data['OrderId'] == order_id, 'ProductId'].unique().tolist()
    
    if len(product_ids) > 1:
        print(f"The product IDs for order ID {order_id} are: {product_ids}")

    # Select the corresponding order time and ConditionVariantID from the sales data based on the order ID
    order_data = sales_data.loc[sales_data['OrderId'] == order_id]
    
    # Generate the query
    query = []

    #check weight of each product
    for product_id in product_ids:
        if (products['ProductId'] == product_id).any():
            weight = products.loc[products['ProductId'] == product_id,'Weight'].iloc[0]
        else:
            weight = avg_weight

        # Get the order time, FromMishZone, FromRenoZone, request_condition, request_num, and requested_no_library_marks
        # for the current product
        order_time = order_data.loc[order_data['ProductId'] == product_id, 'DateOrdered'].iloc[0]
        FromMishZone = order_data.loc[order_data['ProductId'] == product_id, 'FromMishZone'].iloc[0]
        FromRenoZone = order_data.loc[order_data['ProductId'] == product_id, 'FromRenoZone'].iloc[0]

        request_condition = order_data.loc[order_data['ProductId'] == product_id, 'ConditionVariantID'].iloc[0]
        #request_num = order_data.loc[order_data['ProductId'] == product_id, 'Books'].iloc[0]
        request_num = order_data[order_data['ProductId'] == product_id]['Books'].sum()
        requested_no_library_marks = []
        if request_condition in [3, 5, 7, 14]:
            requested_no_library_marks.append(0)
        else:
            requested_no_library_marks.append(1)
        
        # Append the query of the current product to the query list
        query.append({
            'order_id': order_id,
            'product_id': product_id,
            'request_num': request_num,
            'order_time': order_time,
            'request_condition': request_condition,
            'requested_no_library_mark': requested_no_library_marks[0],
            'FromMishZone': FromMishZone,
            'FromRenoZone': FromRenoZone,
            'weight':weight
        })

    return query




In [9]:
query = generate_query(83537217)

No such order ID:83537217


### 1: Inventory filter
For each order we have, the first process is filtering process. We built a filter to get all of the inventory that meet requirement of the order.

In [14]:
# Filter out all the conditionIDs that satisfy the requirement condition
def filter_condition(request_condition):
    condition_w_mark_dict = {
        1: [1],
        14:[1,8,14],
        8: [1, 8],
        5: [1,4,8,14,5],
        4: [1,4,8],
        3: [1,4,8,14,5,2,3],
        2: [1,4,8,2],
        6: [1,4,8,2,6],
        7: [1,4,8,14,5,2,3,6,7]
    }
             
    c_dict = condition_w_mark_dict
    # Returning filter conditions for the requested condition, or an empty list if it does not exist in the dictionary
    return c_dict[request_condition] if request_condition in c_dict else []


'''Determine whether the product_id in the query exists in the inventory data, 
then filter the products whose sales time is between the incoming and outgoing time in the inventory data, 
then filter the products whose quantity and conditionID meet the order requirements, 
and finally return the dataframe containing 'BookFactKey', 'ProductId', 'FulfillmentLocation', 'ConditionVariantID'
'''
def check_single_inventory(queries):
    # Define an empty DataFrame to store the results
    result_df = pd.DataFrame(columns=['BookFactKey', 'ProductId', 'FulfillmentLocation', 'ConditionVariantID'])

    No_inventory = []

    # Looping through each query in the list of queries
    for query in queries:
         # Checking if the product_id in the query exists in the 'ProductId' column of the dataframe
        if query['product_id'] not in list(accepted_inv['ProductId']):
            print(f"{query['product_id']}, no inventory")
            No_inventory.append(query['product_id'])
            continue
        
        # Creating a temporary dataframe with the 'ProductId' column matching the product_id in the query
        else:
            df_temp = accepted_inv[accepted_inv['ProductId'] == query['product_id']]

            # Extracting the order time from the query
            order_time = query['order_time']

            # Filtering the temporary dataframe based on the order time and 'DateProcessed' and 'DateLeftInventory' columns
            df_temp = df_temp[(df_temp['DateProcessed'] <= order_time) &
                    ((df_temp['DateLeftInventory'] +pd.Timedelta(days=2) >= order_time) | 
                        (df_temp['DateLeftInventory'].isna()))]

            # Extracting the request condition from the query and obtaining the corresponding filter conditions using the 'filter_condition' function
            request_condition = query['request_condition']
            condition_code = filter_condition(request_condition)

            # Filtering the temporary dataframe based on the 'ConditionVariantID' column using the filter conditions
            df_temp = df_temp[df_temp['ConditionVariantID'].isin(condition_code)]

            # Checking if the requested number of products is available in the filtered temporary dataframe
            if query['request_num'] > len(df_temp):
                print(f"{query['product_id']} has insufficient stock, existing stock is {len(df_temp)}")
                No_inventory.append(query['product_id'])
                continue
            
            else:
                result_df = pd.concat([result_df, df_temp[['BookFactKey', 'ProductId', 'FulfillmentLocation', 'ConditionVariantID']]],ignore_index=True)

    return result_df, No_inventory



In [ ]:
query= 

### 2: Combination Generator:
When we get all inventory we have, we can get all possible book combinations:

In [6]:
def check_inventory(queries):
    # Call check_single_inventory function to get the results for single query
    q_res, no_inventory = check_single_inventory(queries)

    #Drop the rows that has the same 'ProductId', 'FulfillmentLocation', 'ConditionVariantID'
    q_res.drop_duplicates(subset=['ProductId', 'FulfillmentLocation', 'ConditionVariantID'], inplace=True)
    # Creates a dataframe from the queries
    order_df = pd.DataFrame(queries)

    # If no results were found for the query
    if q_res is None:
        print('No results')
        return pd.DataFrame([])
    
    else:
        # Merge the results of the query with the order dataframe based on ProductId and product_id
        q_res = pd.merge(q_res, order_df, how= 'left', left_on = 'ProductId', right_on = 'product_id')

        # Filter the order dataframe based on the ProductId that was found in the query results
        order_df = order_df[order_df['product_id'].isin(q_res['ProductId'])]

        # Get all possible combinations of BookFactKey for each product
        com_dic = {}
        for i in q_res['ProductId'].unique():
            book_keys = set(q_res[q_res['ProductId'] == i]['BookFactKey'])
            com = list(itertools.combinations(book_keys, int(order_df[order_df['product_id'] == i]['request_num'])))
            com_dic[i] = com   
        # Create a list of lists containing the values from each key
        list_of_lists = [v for v in com_dic.values()]  
        # Get all combinations of the values
        combinations = itertools.product(*list_of_lists)   
        # Flatten the tuples and convert to a single tuple
        ls = []
        for c in combinations:
            temp = ()
            for t in c:
                temp += t
            ls.append(temp)
        # create an empty final dataframe with column names
        final_df = pd.DataFrame(columns=q_res.columns) 
         #Adds a column for the combo number
        final_df['#Combo'] = []
        number = 1

        # Loop through all the possible combinations of BookFactKey values
        for com in ls:
            # Filter the query results based on the BookFactKey values in the current combination
            new_data = q_res[q_res['BookFactKey'].isin(com)]
            # Add the combo number to the filtered results
            new_data['#Combo'] = number
            # Concatenate the filtered results with the final dataframe
            final_df = pd.concat([final_df,new_data])
            number += 1
            if new_data['FulfillmentLocation'].nunique() == 1:
                final_df = new_data
                break

    # Drop the product_id column from the final dataframe
    final_df = final_df.drop('product_id', axis=1)

    return final_df,no_inventory

In [10]:
queries = generate_query(83537217)


No such order ID:83537217


### 3: Shipping Cost
Then we can calculate shipping cost of each possible combination based on its weight, fulfullment location and domestic zone:

In [6]:
def ship_rate_book(weight, dzone):
    # Round a number to only one decimals:
    weight = round(weight, 1)
    try:
        # check the shipping rate of a single book based on its weight and domestic zone
        rate = shipping_rate.loc[(shipping_rate['WeightMin'] <= weight)&
                      (shipping_rate['WeightMax'] >= weight)&
                      (shipping_rate['DomesticZone'] == dzone),'ShippingRate'].values[0]
    except IndexError:
        rate = 0
        
    return rate

In [7]:
def shipping_cost_calculator(data):
    data['ShippingVariable'] = data.apply(lambda x : 1 if x['FulfillmentLocation'] == 'Mish' else -1, axis= 1)
    # get the weight and dometstic zone of different fulfilment location of each combo
    weight_df = data.groupby(['#Combo','FulfillmentLocation']).aggregate({'weight': sum, 'FromMishZone': np.mean,  'FromRenoZone': np.mean, 'ShippingVariable' : np.mean}).reset_index()
    
    # calcuate the shipping rate from Mish and Reno seperately
    weight_df['ShippingCostMish'] = weight_df.apply(lambda x: ship_rate_book(x['weight'], x['FromMishZone']), axis = 1)
    weight_df['ShippingCostReno'] = weight_df.apply(lambda x: ship_rate_book(x['weight'], x['FromRenoZone']), axis = 1)

    # choose if the shipping rate is 'shippingcostmish' or 'shippingcostreno' based on its fulfillmentlocation
    weight_df['ShippingCost'] = weight_df.apply(lambda x: x['ShippingCostMish'] if x['FulfillmentLocation'] == 'Mish' else x['ShippingCostReno'], axis= 1)

    # sum shipping rate from different places of a combination
    ship_cost = weight_df.groupby('#Combo').aggregate({'ShippingCost': sum, 'ShippingVariable': sum}).reset_index()

    return ship_cost

### 4: Condition Cost
 Furthermore, we can calculate condition cost of each possible combination based on the gap between required condtion and the real condition of each book in the combo.

In [8]:
def condition_cost_calculator(data):

    unique_bookIDs = np.unique(data['BookFactKey'])

    x = 0
    while x < len(unique_bookIDs):
        single_book = data[data['BookFactKey'] == unique_bookIDs[x]].iloc[0]
        # taking the first instance of bookID since all are same and I only want to work with one

        # library mark cost
        if single_book['requested_no_library_mark'] == 0 and single_book['ConditionVariantID'] in [1,8,4,2,6]: 
            # For books that didn't care if you gave a book without lib markings, but the book does not have lib markings.
            libcost = 0.10 # cost of upgrading to a book without a library marking
        else: 
            libcost = 0

        # condition upgrade cost

        # where in the condition dataframe is the condition that the book picked is
        book_cond_row = cond_cost_df[cond_cost_df['Condition'].str.contains(str(single_book['ConditionVariantID']),case=False)].index[0]
        # where in the condition dataframe is the condition that the order asked for
        req_cond_row = cond_cost_df[cond_cost_df['Condition'].str.contains(str(single_book['request_condition']),case=False)].index[0]

        #if book_cond_row < req_cond_row:
        #print(book_cond_row, req_cond_row, single_book['ConditionVariantID'], single_book['request_condition'])

        # calculate cost of upgrade
        book_cond_up_cost = sum(cond_cost_df['Cost'][book_cond_row+1:req_cond_row+1]) # upgrading from asking to book using cost dataframe

        # total condition cost for book
        book_cond_cost_num = libcost + book_cond_up_cost

        # adding that cost to that bookID for each combination it appears in.
        data.loc[data['BookFactKey'] == single_book['BookFactKey'], 'book_cond_cost'] = book_cond_cost_num

        x += 1

    return data

When we have the shipping cost and condition cost of each combo, we can get the optimizated combo in terms of the lowest shipping cost and lowest total cost:

In [9]:
def lowest_shipping_generator(condition_cost_df, ship_cost_df):
    combo_df = condition_cost_df.groupby(['#Combo'])[['book_cond_cost']].sum().reset_index() # shows the summed combo cost for each combination

    # inner join condition cost data frame and condition data frame
    combo_df = pd.merge(combo_df, ship_cost_df, how='inner', on = '#Combo')

    # get total cost by adding condition cost data to condition cost
    combo_df['totalcost'] = combo_df['book_cond_cost'] + combo_df['ShippingCost']
    
    # Get the best combination based on shipping cost
    min_s_index = combo_df['ShippingCost'].idxmin()
    min_shipping = pd.DataFrame(combo_df.loc[min_s_index][['#Combo','ShippingCost','totalcost','ShippingVariable']]).T.reset_index()
    min_shipping.rename(columns= {'#Combo':'LowestShippingCostCombo','totalcost':'TotalCost1', 'ShippingVariable':'ShippingVariable1'}, inplace= True)
    
    # Get the best combination based on total cost
    min_t_index = combo_df['totalcost'].idxmin()
    min_total = pd.DataFrame(combo_df.loc[min_t_index][['#Combo','totalcost', 'ShippingVariable']]).T.reset_index()
    min_total.rename(columns= {'#Combo':'LowestTotalCostCombo', 'totalcost':'TotalCost2','ShippingVariable':'ShippingVariable2'}, inplace= True)

    #Concatenate the lowest shipping cost combo with the lowest total cost combo
    final_df = pd.concat([min_shipping, min_total], axis=1).drop(columns=['index'])

    return final_df


In [10]:
## Get the shipping cost based on their CURRENT model of BWB
def get_previous_cost(order_num, to_remove):
    sd = sales_data[sales_data['OrderId'] == order_num]
    #remove product that is out of inventory
    sd = sd[~sd['ProductId'].isin(to_remove)]
    #sd['TotalPostage'] = sd['Postage'] * sd['Books']
    #previous_cost = sd['TotalPostage'].sum()

    #calculate shipping rate by zone and weight
    weight = sd.groupby('ShipFromWarehouse').aggregate({'Weight': sum, 'FromMishZone': np.mean,  'FromRenoZone': np.mean}).reset_index()
    
    # calcuate the shipping rate from Mish and Reno seperately
    weight['ShippingCostMish'] = weight.apply(lambda x: ship_rate_book(x['Weight'], x['FromMishZone']), axis = 1)
    weight['ShippingCostReno'] = weight.apply(lambda x: ship_rate_book(x['Weight'], x['FromRenoZone']), axis = 1)

    # choose if the shipping rate is 'shippingcostmish' or 'shippingcostreno' based on its fulfillmentlocation
    weight['ShippingCost'] = weight.apply(lambda x: x['ShippingCostMish'] if x['ShipFromWarehouse'] == 'Mish' else x['ShippingCostReno'], axis= 1)
    previous_cost = weight['ShippingCost'].sum()
    
    return previous_cost

### 5: Final Optimization Model:
the whole process we went through is only applicable for a sinle order.
So, we built the final optimization model to get the optimizated combination of mutiple orders in the end.

In [11]:
def order_optimization(order_ls):
    
    optimized_df = pd.DataFrame([])

    # loop every order in order_ls and get its optimizated combination
    for i in range(len(order_ls)):

        order_num = order_ls[i]
        # get possible inventory
        queries = generate_query(order_num)
        
        #Limit the number of books to less than 10
        if len(queries) > 10:
            continue
        # only test orders which have more than one books
            
        # all possible combinations
        data,no_inventory = check_inventory(queries)
        if data.empty == True:
            continue
        
        data
        # Calculate ship cost of each combination
        ship_cost = shipping_cost_calculator(data)
        # Calculate condition cost of each combination
        condition_cost = condition_cost_calculator(data)
        #get the optimized combination
        optimized_single = lowest_shipping_generator(condition_cost, ship_cost)
        optimized_single['OrderId'] = order_num
        #get previous cost
        optimized_single['previous_cost'] = get_previous_cost(order_num,no_inventory)

        optimized_df = pd.concat([optimized_df, optimized_single])

        if i % 500 == 0:
            optimized_df.to_csv(f'LatestResultWith{i}Orders.csv')

    #columns = optimized_df.columns.tolist()
    #columns = columns[-1:] + columns[:-1]

    # reorder the columns
    #optimized_df = optimized_df[columns].reset_index().drop(columns=['index'])

    return optimized_df


### Test Order

In [12]:
# get test order list
order_ls = list(sales_data['OrderId'].unique()[-15000:])

In [ ]:
# get final results
import warnings
warnings.filterwarnings('ignore', '.*')

optimized_df = order_optimization(order_ls)

The product IDs for order ID 80557116 are: [173051, 1506230]
The product IDs for order ID 80557123 are: [3474541, 5838995]
The product IDs for order ID 80557185 are: [170397, 22066, 160741]
The product IDs for order ID 80557194 are: [29047628, 10521527]
1026217 has insufficient stock, existing stock is 1
The product IDs for order ID 80557217 are: [12648384, 3735125, 5648605, 25253586, 1262829, 6316087, 5991229, 1100811]
3735125 has insufficient stock, existing stock is 0
25253586 has insufficient stock, existing stock is 0
5991229 has insufficient stock, existing stock is 0
The product IDs for order ID 80557218 are: [127326, 118632, 14210, 125722]
The product IDs for order ID 80557219 are: [587006, 32770809, 1093342, 25133839, 634700, 34203620, 1053914]
The product IDs for order ID 80557221 are: [26516272, 30589924, 29897638, 2695034, 50386, 4762618, 6997679, 459738, 123210]
The product IDs for order ID 80557223 are: [33337, 571844, 1441287, 105594]
The product IDs for order ID 8055722

The product IDs for order ID 80558110 are: [6590428, 2621749, 273099, 365994, 938162]
The product IDs for order ID 80558111 are: [32008038, 30491006, 21473181]
The product IDs for order ID 80558114 are: [40074333, 33369634, 5441147, 24887991]
The product IDs for order ID 80558122 are: [245682, 869978, 209241]
The product IDs for order ID 80558126 are: [50498, 1338974, 1500286]
1500286 has insufficient stock, existing stock is 0
The product IDs for order ID 80558130 are: [12350128, 1553669, 1383871]
12350128 has insufficient stock, existing stock is 0
The product IDs for order ID 80558138 are: [1605653, 1605465, 25518068, 143333]
The product IDs for order ID 80558144 are: [930445, 26112276]
930445 has insufficient stock, existing stock is 0
26112276 has insufficient stock, existing stock is 0
The product IDs for order ID 80558157 are: [10849779, 1354194, 109464, 31389733, 1499630]
The product IDs for order ID 80558160 are: [23652797, 28083161, 25528001]
The product IDs for order ID 8055

1484831, no inventory
The product IDs for order ID 80559049 are: [144467, 1750738]
1750738 has insufficient stock, existing stock is 0
25239707 has insufficient stock, existing stock is 0
The product IDs for order ID 80559057 are: [579429, 28717270, 38016996, 30408625, 34251562]
34251562 has insufficient stock, existing stock is 0
The product IDs for order ID 80559062 are: [2032722, 2153999, 615533]
2032722 has insufficient stock, existing stock is 0
2153999 has insufficient stock, existing stock is 0
615533 has insufficient stock, existing stock is 0
The product IDs for order ID 80559063 are: [13737696, 280054, 1771222, 32004979]
The product IDs for order ID 80559064 are: [3812088, 25659889]
3812088 has insufficient stock, existing stock is 0
25659889 has insufficient stock, existing stock is 0
The product IDs for order ID 80559162 are: [1658475, 32540964, 181317, 46417591, 1313384, 576164, 38197066]
1658475 has insufficient stock, existing stock is 0
32540964 has insufficient stock, 

The product IDs for order ID 80560390 are: [1562094, 2306589, 50786589, 8956184, 12057236]
1562094 has insufficient stock, existing stock is 0
2306589, no inventory
The product IDs for order ID 80560391 are: [21726287, 25659032]
The product IDs for order ID 80560459 are: [39160612, 33508056]
The product IDs for order ID 80560546 are: [46768247, 40691741]
The product IDs for order ID 80560591 are: [23691737, 28889544, 2314205, 22774204, 34256221, 1932777]
34256221 has insufficient stock, existing stock is 0
The product IDs for order ID 80560593 are: [1102598, 11041725, 27431081, 866670, 2337693]
11041725 has insufficient stock, existing stock is 0
The product IDs for order ID 80560596 are: [3669329, 39084, 1440379, 23281142, 293367, 22780233, 24259509, 7775272, 5694943, 838609]
22780233 has insufficient stock, existing stock is 0
The product IDs for order ID 80560611 are: [890394, 19924]
890394 has insufficient stock, existing stock is 0
The product IDs for order ID 80560652 are: [19497

1671954 has insufficient stock, existing stock is 0
The product IDs for order ID 80562276 are: [8757695, 21244767, 13062108]
The product IDs for order ID 80562279 are: [25007517, 2275230]
The product IDs for order ID 80562294 are: [21957889, 26866383, 31111, 1308291, 24437337, 1625629, 176184, 29602945, 22952267]
The product IDs for order ID 80562296 are: [21719444, 27664740, 4702218, 7009926, 34220589, 616572, 6021321, 807170]
4702218, no inventory
6021321 has insufficient stock, existing stock is 0
The product IDs for order ID 80562305 are: [8329204, 27664702, 30982187, 34178911, 25108142]
The product IDs for order ID 80562353 are: [39475541, 40692680]
The product IDs for order ID 80562425 are: [2017852, 1568579]
1568579 has insufficient stock, existing stock is 0
The product IDs for order ID 80562449 are: [428742, 3651755]
3651755 has insufficient stock, existing stock is 0
The product IDs for order ID 80562451 are: [34383084, 23174714, 41863324]
23174714 has insufficient stock, exi

...

The product IDs for order ID 80698658 are: [382359, 1439783]
The product IDs for order ID 80698660 are: [26274906, 18980]
The product IDs for order ID 80698663 are: [16729234, 112683, 3551342, 23776454, 16663249, 21935653, 37401083]
112683 has insufficient stock, existing stock is 0
The product IDs for order ID 80698664 are: [571249, 3013060, 1604740]
The product IDs for order ID 80698667 are: [23341313, 33697121, 7343378, 255969, 1482745, 21516839, 25141874]
33697121 has insufficient stock, existing stock is 0
The product IDs for order ID 80698679 are: [33366654, 34252666, 27159609]
The product IDs for order ID 80698696 are: [32993024, 2288172]
The product IDs for order ID 80698698 are: [26146654, 1519296, 11763042, 2090994, 2900991, 5900255, 1404475]
The product IDs for order ID 80698699 are: [34790170, 30982636, 25012118, 1782215]
The product IDs for order ID 80698701 are: [1424129, 1665295, 1768972, 240379]
1424129 has insufficient stock, existing stock is 0
1665295 has insufficien

22192310, no inventory
30785182 has insufficient stock, existing stock is 0
The product IDs for order ID 80699478 are: [1970625, 42379830]
The product IDs for order ID 80699480 are: [24259509, 43965104, 41098377]
The product IDs for order ID 80699482 are: [51442684, 50012532, 1614565, 51269583, 48727273]
The product IDs for order ID 80699483 are: [1622410, 3668159, 122626]
The product IDs for order ID 80699485 are: [161867, 41351192, 33448048, 1332330, 33934513, 1300823, 512006, 1336997, 5542601, 1694250, 574770, 772940, 726474, 5585732, 11542939]
The product IDs for order ID 80699487 are: [24101068, 32879960, 24192464, 28290524, 32879426, 32879975]
The product IDs for order ID 80699490 are: [87538, 42180380, 32342917, 20791474]
The product IDs for order ID 80699491 are: [22243305, 23696291, 3578496, 7091357]
The product IDs for order ID 80699492 are: [48568862, 33794314, 1459625, 31492350, 1294648]
1459625 has insufficient stock, existing stock is 0
The product IDs for order ID 806994

The product IDs for order ID 80700036 are: [3818266, 21927305, 23815994, 3818267, 1567469, 21927026]
The product IDs for order ID 80700038 are: [36908843, 5833078, 28716990]
The product IDs for order ID 80700049 are: [39571532, 46284163]
The product IDs for order ID 80700051 are: [746953, 1360975, 388640, 3388004]
The product IDs for order ID 80700062 are: [575401, 624982, 1938859, 2250248, 277531]
The product IDs for order ID 80700064 are: [43342183, 20667086]
The product IDs for order ID 80700068 are: [8429730, 8205313, 31001823, 37587822, 1442461, 27603458, 367006, 1143650]
27603458 has insufficient stock, existing stock is 0
The product IDs for order ID 80700071 are: [51155578, 50162884, 51294610]
The product IDs for order ID 80700072 are: [22879940, 28145389, 25780596, 21716949]
21716949 has insufficient stock, existing stock is 0
The product IDs for order ID 80700073 are: [15738946, 188497, 889665, 22444144, 11219648, 28135878, 28788821]
15738946 has insufficient stock, existing 

The product IDs for order ID 80700446 are: [620885, 527043, 1456095, 709678]
The product IDs for order ID 80700447 are: [2226205, 1373322, 3161971, 252178, 1175684, 1360886, 3637586, 10732517, 3425608]
1373322 has insufficient stock, existing stock is 0
The product IDs for order ID 80700452 are: [898552, 38022171, 108404, 2006836, 53936, 1456901]
The product IDs for order ID 80700453 are: [16347726, 6422115, 92866, 3462560, 32765664, 428971]
16347726 has insufficient stock, existing stock is 0
The product IDs for order ID 80700456 are: [564120, 26655043, 65114, 2681236]
The product IDs for order ID 80700458 are: [28153061, 735995, 24233526, 1324399, 2656080, 2660172, 1665775, 52509]
28153061 has insufficient stock, existing stock is 0
2660172, no inventory
The product IDs for order ID 80700459 are: [48614029, 1766757, 9370029, 18247287, 777496, 22975104, 37055886, 32649886, 780584, 40692669, 801216, 420558, 50289135, 1697906, 882233, 584653, 12321482, 543930, 25634001, 17020096, 213690

201591 has insufficient stock, existing stock is 0
The product IDs for order ID 80700875 are: [34237040, 51170262, 495707, 47699282, 25305753, 1485860]
34237040 has insufficient stock, existing stock is 0
The product IDs for order ID 80700879 are: [2190314, 3622637, 1496246, 1496247]
The product IDs for order ID 80700880 are: [12988847, 437715, 51196362, 6946634]
12988847 has insufficient stock, existing stock is 0
The product IDs for order ID 80700881 are: [1455981, 78829, 12104598, 41388281, 11538307, 99425, 31900688, 26596280, 10055832, 25756864]
The product IDs for order ID 80700886 are: [2098635, 1761785, 275991, 15623551, 1192185, 622268, 93555, 21987499, 50028496, 46582206, 33034122, 12869309, 1670385, 3139002, 376712, 7423209, 227872, 1032342, 1341259, 1179220]
The product IDs for order ID 80700893 are: [28663245, 22375152, 22791865]
The product IDs for order ID 80700909 are: [9001675, 81199, 21376075, 21489, 826600, 85345, 2455020, 1502220, 404922]
85345 has insufficient stock

In [ ]:
optimized_df.to_csv('Final_Result.csv')